<a href="https://colab.research.google.com/github/Jungkihong07/Gptrip/blob/main/text_embedding_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 임베딩


## 1.   본 코드의 목적
  

    *   프로젝트의 목적에 맞는 테스트셋 변형
    *   테스트 셋 임베딩
    *   FAISS를 통한 유사도 확인



# 필요 라이브러리 설치

In [1]:
!pip install -U sentence-transformers tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 664.8/664.8 MB 90.5 MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [6]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.4 MB/s eta 0:00:00


In [1]:
import numpy as np
import torch
import faiss

print("✅ NumPy 버전:", np.__version__)
print("✅ PyTorch 버전:", torch.__version__)
print("✅ FAISS 사용 가능:", hasattr(faiss, "IndexFlatL2"))

✅ NumPy 버전: 1.26.4
✅ PyTorch 버전: 2.6.0+cu124
✅ FAISS 사용 가능: True


# 모델 로드 및 디바이스 설정

In [2]:
import pandas as pd
import numpy as np
import faiss
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# ✅ 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ 현재 사용 디바이스: {device}")

# ✅ 모델 로드
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", device=device)


✅ 현재 사용 디바이스: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 데이터 셋 임베딩

In [9]:

# ✅ 감성 요약 불러오기
df = pd.read_csv("incheon_with_emotional_summary_for_recommender_2.csv")
sentences = df["emotional_summary"].dropna().tolist()

# ✅ tqdm으로 진행률 출력하면서 임베딩 생성
print("🔄 장소 임베딩 생성 중...")
embeddings = []
for sentence in tqdm(sentences, desc="Embedding 장소"):
    emb = model.encode(sentence, convert_to_tensor=True, device=device)
    embeddings.append(emb)

# ✅ 텐서를 numpy로 변환
embeddings_tensor = torch.stack(embeddings)
embeddings_np = embeddings_tensor.cpu().detach().numpy()

# ✅ 임베딩 저장 (.npy)
np.save("incheon_place_embeddings.npy", embeddings_np)
print("✅ 임베딩 파일 저장 완료: incheon_place_embeddings.npy")

# ✅ FAISS 인덱스 생성 및 저장
dim = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings_np)
faiss.write_index(index, "incheon_faiss_index.index")
print("✅ FAISS 인덱스 저장 완료: incheon_faiss_index.index")


🔄 장소 임베딩 생성 중...


Embedding 장소: 100%|██████████| 461/461 [00:05<00:00, 82.55it/s]

✅ 임베딩 파일 저장 완료: incheon_place_embeddings.npy
✅ FAISS 인덱스 저장 완료: incheon_faiss_index.index


## 다시 불러올 때

In [ ]:
# 불러오기
import numpy as np
import faiss

embedding_np = np.load("incheon_place_embeddings.npy")
index = faiss.read_index("incheon_faiss_index.index")

# 정확도 측정
- 정확도를 측정하기 위해, 상대적인 조건을 확인한다. 사용자 입장에서의 맥락에 맞는 유사성을 확인한다.
=> 확인 결과 어느 정도의 유사성을 확인함.

In [12]:

# ✅ 사용자 입력 임베딩 + 검색
user_input = "가족들과 갈 수 있는 한적한 호수"
print(f"\n📝 사용자 입력: {user_input}")
user_vec = model.encode([user_input], convert_to_tensor=True, device=device)
user_vec_np = user_vec.cpu().detach().numpy()

# ✅ FAISS 검색
top_k = 5
_, indices = index.search(user_vec_np, top_k)

# ✅ 결과 출력
print(f"\n🎯 추천 결과 (Top-{top_k}):")
result = df.iloc[indices[0]][["title", "emotional_summary"]]
print(result.to_string(index=False))



📝 사용자 입력: 가족들과 갈 수 있는 한적한 호수

🎯 추천 결과 (Top-5):
   title                                                                                           emotional_summary
  인천중앙공원 "도심 속에서 잔잔한 호수를 따라 산책하며 계절의 변화를 느낄 수 있고, 다양한 문화 행사와 축제를 즐길 수 있는 이곳은 여유롭게 휴식을 취하며 가족과 함께하는 시간을 풍요롭게 만들어줍니다."
   동검선착장                                        "차박 캠핑의 편리함을 느낄 수 있는 곳에서 서해안의 갯벌 체험과 낚시를 즐기며, 가족과 함께 고요한 시간을 보내보세요."
      목섬                                 썰물이 드러내는 모랫길을 따라 걸으며, 모세의 기적을 경험할 수 있는 고요한 섬에서 가족과 함께 갯벌 체험과 어촌 생활을 만끽해보세요.
    아라빛섬             조용한 호수 위에 위치한 이 곳에서는 서해의 바다를 향해 산책을 즐기며, 인천의 전경을 한눈에 담는 전망대에서 마치 판타지 속에 들어온 것 같은 느낌을 받을 수 있습니다.
후포항 선수포구                           작은 어촌 마을의 조용한 분위기 속에서 갯벌을 탐험하고, 신선한 밴댕이를 맛보며 해안도로를 따라 일몰을 감상하는 곳에서 조용한 여유를 즐겨보세요.
